# Hồi quy tuyến tính
*Hoàn thành toàn bộ phần bài tập trong notebook này, bao gồm toàn bộ kết quả đầu ra và code hỗ trợ.*

***"Không có một sự kiện nào trên đời là ngẫu nhiên, những thứ đang cho là ngẫu nhiên chỉ là những sự kiện ta chưa tìm ra được mô hình để biểu diễn quy luật của chúng".***

Xây dựng mô hình **Hồi quy tuyến tính** bao gồm hai phần:
- Trong quá trình huấn luyện, bộ phân lớp lấy dữ liệu huấn luyện và và học các tham số mô hình.
- Trong quá trình kiếm tra, mô hình phân lớp từng đối tượng bằng cách nhân giá trị của mẫu với các tham số mô hình để tìm ra giá trị của nhãn.
- Giá trị của tham số được kiểm định chéo.
Trong bài tập này, bạn sẽ cài đặt những bước trên và hiểu được qui trình Xây dựng một mô hình đơn giản với Học tham số, kiểm định chéo, và hiểu được cách viết code hiệu quả với vectorize.

Bài toán dự đoán giá nhà Boston được sử dụng trong bài tập này.

In [ ]:
# Import một số thư viện cần thiết.
import random
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split 
import matplotlib.pyplot as plt

# Sử dụng một mẹo nhỏ để vẽ hình trên cùng một dòng thay vì mở cửa sổ mới
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # đặt kích thước mặc định cho hình
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Một mẹo nhỏ để notebook tự load lại các module bên ngoài;
# xem thêm tại http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
# Tải dữ liệu Giá nhà Boston từ Scikit-learn.
boston = datasets.load_boston()
X_train, X_test, y_train, y_test = train_test_split(boston.data, \
                                                    boston.target, test_size=0.2)

# As a sanity check, we print out the size of the training and test data.
print('Training data shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

## Load dữ liệu
Hồi qui tuyến tính đơn giản là một cách tiếp cận để dự đoán phản ứng (giá trị đầu ra) khi dữ liệu có một đặc trưng duy nhất. Khi giả sử hai biến $x$ và $y$ liên hệ tuyến  tính thì mục tiêu của mô hình là cố tìm ra đường tuyến tính tốt nhất để dự đoán phản ứng ($y$). 

Đường đó được gọi là đường hồi quy.

Công thức cho đường hồi quy được biểu diễn như sau:
$$ \hat{Y} = h(X) = XW$$
Trong đó: 

- $X$ là ma trận có kích thước $N \times D$ với $X_{ij}$ là giá trị của đặc trưng thứ $j$ của mẫu $i$.
- $W$ là ma trận tham số có kích thước $D \times 1$
- $Y$ là giá trị phản ứng của $N$ mẫu.



In [ ]:
# Biểu diễn một số ví dụ trong tập huấn luyện sử dụng một đặc trưng duy nhất.
# LSTAT - % lower status of the population
plt.scatter(X_train[:,12], y_train)
plt.xlabel("Crime rate")
plt.ylabel("House's price")
plt.show()

## Huấn luyện mô hình
Tất cả code cho phần bài tập này được lưu trong tệp **models/linear_regression.py** và **models/linear_loss.py**.
### Cập nhật tham số
Quá trình huấn luyện mô hình thực chất là từ dữ liệu để học ra tham số mô hình phù hợp nhất với mô hình sinh dữ liệu. Trong mô hình hồi quy tuyến tính, ta cần học tham số $W$.

Khi khởi tạo mô hình, ta giả sử tham số được khởi tạo ngẫu nhiên. Sử dụng tham số $W$ đó, ta ước lượng được giá trị $Y$:
$$ \hat{y} = h(X) = WX $$

Tổng sai số, độ lệch của giá trị dự đoán so với giá trị thực tế gọi là hàm giá trị (Cost function):
$$ J(w) = \frac{1}{2N}\sum_{i=1}^{N} (\hat{y}_i - y_i)^2 = \frac{1}{2N}\sum_{i=1}^{N}\sum_{j=1}^{D} (w_{j}x_{ij} - y_{ij})^2$$


Chúng ta sử thuật toán **xuống đồi (Gradient descent)** để tối ưu tham số $W$. (Xem khóa [Machine Learning](https://www.coursera.org/learn/machine-learning/))

Đột tụt dốc của tham số $W$ được cập nhật theo công thức:
$$ dw_i = \frac{\partial}{\partial w_i}J(w)$$

Đầu tiên, mở file ```models/linear_loss.py``` và cài đặt hàm ```linear_loss_naive```, sử dụng vòng lặp để tính hàm giá trị (Cost function).

In [ ]:
from models.linear_loss import linear_loss_naive
import time

# sinh ngẫu nhiên các trọng số (W) với các giá trị nhỏ
W = np.random.randn(13, ) * 0.0001 

loss, grad = linear_loss_naive(W, X_test, y_test, 0.00001)
print('loss: %f' % (loss, ))

Lúc này, các giá trị gradient được trả về đều bằng 0. Đạo hàm và tính gradient theo công thức được cho ở trên trong cùng hàm ```linear_loss_naive```. Bạn sẽ thấy một số thứ hữu ích trong phần cài đặt trước đó.

Để đảm bảo là bạn đã cài đặt đúng, chúng ta sẽ sử dụng hàm ```grad_check_sparse``` (đã được cài đặt sẵn) để kiểm tra.

In [ ]:
# Bởi vì bạn đã cài đặt hàm gradient, tính toán gradient với code dưới đây và
# kiểm tra với hàm grad_check_sparse(...) đã cho.

# Tính toán loss và grad với W.
loss, grad = linear_loss_naive(W, X_test, y_test, 0.0)

# Tính toán gradient theo một số chiều ngẫu nhiên và so sánh chúng với kết quả
# của bạn. Giá trị phải gần như chính xác theo tất cả các chiều.
from models.gradient_check import grad_check_sparse
f = lambda w: linear_loss_naive(w, X_test, y_test, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, grad)

# thực hiện kiểm tra khi có sử dụng regularization
# đừng quên cài đặt gradient với regularization nhé.
loss, grad = linear_loss_naive(W, X_test, y_test, 1e2)
f = lambda w: linear_loss_naive(w, X_test, y_test, 1e2)[0]
grad_numerical = grad_check_sparse(f, W, grad)

# Kết quả relative error trong khoảng 1e-12

In [ ]:
# Kế tiếp, cài đặt linear_loss_vectorized; hiện tại chỉ tính toán hàm giá trị;
# gradient sẽ cài đặt sau.
tic = time.time()
loss_naive, grad_naive = linear_loss_naive(W, X_test, y_test, 0.00001)
toc = time.time()
print('Naive loss: %e computed in %fs' % (loss_naive, toc - tic))

# Vectorized
from models.linear_loss import linear_loss_vectorized
tic = time.time()
loss_vectorized, _ = linear_loss_vectorized(W, X_test, y_test, 0.00001)
toc = time.time()
print('Vectorized loss: %e computed in %fs' % (loss_vectorized, toc - tic))

# Hàm giá trị khi vectorized nên có cùng giá trị với giá trị được tính bằng hàm
# linear_loss_naive() nhưng tính toán nhanh hơn
print('difference: %f' % (loss_naive - loss_vectorized))

In [ ]:
# Hoàn thiện phần cài đặt của linear_loss_vectorized, và tính toán gradient theo
# cách vectorized.

# Hai hàm tính loss và gradient nên cho kết quả giống nhau nhưng bản vectorized 
# tính toán nhanh hơn.
tic = time.time()
_, grad_naive = linear_loss_naive(W, X_test, y_test, 0.00001)
toc = time.time()
print('Naive loss and gradient: computed in %fs' % (toc - tic))

tic = time.time()
_, grad_vectorized = linear_loss_vectorized(W, X_test, y_test, 0.00001)
toc = time.time()
print('Vectorized loss and gradient: computed in %fs' % (toc - tic))

# So sánh gradient
difference = np.linalg.norm(grad_naive - grad_vectorized)
print('difference: {}'.format(difference))

### Huấn luyện với hàm cập nhật
Sử dụng các hàm ```loss``` đã cài đặt ở trên để cài đặt hàm ```train``` trong tệp **linear_regression.py**.

Tham số W được cập nhật từng thành phần theo công thức:
$$ w_i =  w_i -\alpha\frac{\partial}{\partial w_i}J(w)$$

In [ ]:
# Ở trong tệp linear_regression.py, cài đặt hàm LinearRegression.train() và chạy
# hàm đó với code sau
from models.linear_regression import LinearRegression
clf = LinearRegression()
tic = time.time()
loss_hist = clf.train(X_train, y_train, learning_rate=1e-7, reg=5e4,
                      num_iters=1500, verbose=True)
toc = time.time()
print('That took %fs' % (toc - tic))

In [ ]:
# Một chiến thuật debug hiệu quả được sử dụng đó là vẽ ra lịch sử mất mát (loss 
# history) như là một hàm với số lần lặp.
plt.plot(loss_hist)
plt.xlabel('Iteration number')
plt.ylabel('Loss value')
plt.show()

In [ ]:
# Cài đặt hàm LinearRegression.predict đánh giá hiệu năng mô hình trên cả tập
# huấn luyện và tệp kiểm tra.
y_train_pred = clf.predict(X_train)

# TODO: Tính toán hệ số tương quan của đầu ra mô hình và kết quả thực sự
train_coeff = None
print('training coefficient: %f' % (train_coeff))
y_test_pred = clf.predict(X_test)
test_coeff = None
print('validation coefficient: %f' % (test_coeff))

### Sự chính quy hóa (Regularization)

Regularization, một cách cơ bản, là thay đổi mô hình một chút để tránh overfitting trong khi vẫn giữ được tính tổng quát của nó (tính tổng quát là tính mô tả được nhiều dữ liệu, trong cả tập training và test).

Có rất nhiều phương pháp được sử dụng để chính quy hóa một mô hình, trong đó, chúng ta sẽ tìm hiểu về phương pháp $L_2 \, regularization$.

#### Regularized loss function
Kỹ thuật regularization phổ biến nhất là thêm vào hàm mất mát một số hạng nữa. Số hạng này thường dùng để đánh giá độ phức tạp của mô hình. Số hạng này càng lớn, thì mô hình càng phức tạp. Hàm mất mát mới này thường được gọi là `regularized loss function`, thường được định nghĩa như sau:
$$ J_{reg}(\theta) = J(\theta) + \lambda R(\theta)$$

Trong đó:
- $\theta$ là tham số mô hình (trong trường hợp này là $w$)
- $\lambda$ là hằng số regularization
- $R(\theta)$ là số hạng regularization

Kỹ thuật `l2 regularization`sử dụng $R(\theta) = R(w) = ||w||_2^2$.

#### Cập nhật tham số
Do tham số được cập nhật dựa trên đạo hàm của hàm mất mát, ta cần xây dựng lại công thức tính đạo hàm theo cách sau:
$$ w_i =  w_i -\alpha\frac{\partial}{\partial w_i}J_{reg}(w)$$

** Bài tập:** Dựa trên 2 công thức chuẩn hóa có ở trên, viết lại code cho bộ phân lớp `Linear Regression` và chạy tiếp code sau để tìm được mô hình tối ưu.

In [ ]:
# Sử dụng tập kiểm tra để điều chỉnh các siêu tham số (độ lớn của reg và tỉ
# lệ học. Bạn nên thực nghiệm với nhiều khoảng giá trị của 2 siêu tham số này
learning_rates = [1e-7, 5e-5]
regularization_strengths = [5e4, 1e5]

# kết quả là một từ điển ánh xạ từ tuple có dạng (reg, lr) sang tuple có dạng
# (train_coeff, test_coeff). Độ chính xác chỉ đơn giản là tỉ lệ mẫu dự đoán chính
# xác trên toàn tập dữ liệu.
results = {}
best_coeff = -1    # Hệ số tương quan tốt nhất mà chúng ta sẽ đạt được.
best_linear = None # Mô hình LinearRegression có hiệu năng tốt nhất.

################################################################################
# TODO:                                                                        #
# Viết code chọn các siêu tham số tốt nhất bằng cách điều chỉnh trên tập kiểm  #
# tra. Với mỗi tổ hợp siêu tham số, huấn luyện một mô hình LinearRegression    #
# trên tập huấn luyện, tính toán độ tương quan trên tập huấn luyện và tập kiểm #
# tra, và lưu những con số này vào từ điển kết quả. Thêm vào đó, lưu hiệu năng #
# tốt nhất trên tập kiểm tra vào best_val và mô hình LinearRegression tương    #
# ứng vào best_svm.                                                            #  
#                                                                              #
# Gợi ý: Bạn nên sử dụng số vòng lặp (num_iters) nhỏ khi xây dựng code kiểm    #
# tra để mô hình không mất quá nhiều thời gian để huấn luyện. Khi đã chắc chắn,#
# bạn nên trả về kết quả với số vòng lặp lớn                                   #
################################################################################
pass
################################################################################
#                              KẾT THÚC                                        #
################################################################################
    
# In kết quả
for lr, reg in sorted(results):
    train_coeff, test_coeff = results[(lr, reg)]
    print('lr %e reg %e train coefficient: %f val coefficient: %f' % (
                lr, reg, train_coeff, test_coeff))
    
print('best validation coefficient achieved during cross-validation: %f' % best_test)

In [ ]:
# Visualize kết quả kiểm thử chéo
import math
x_scatter = [math.log10(x[0]) for x in results]
y_scatter = [math.log10(x[1]) for x in results]

# plot training accuracy
marker_size = 100
colors = [results[x][0] for x in results]
plt.subplot(2, 1, 1)
plt.scatter(x_scatter, y_scatter, marker_size, c=colors)
plt.colorbar()
plt.xlabel('log learning rate')
plt.ylabel('log regularization strength')
plt.title('Boston training accuracy')

# vẽ hiệu năng trên tập kiểm tra
colors = [results[x][1] for x in results] # kích thước mặc định của marker là 20
plt.subplot(2, 1, 2)
plt.scatter(x_scatter, y_scatter, marker_size, c=colors)
plt.colorbar()
plt.xlabel('log learning rate')
plt.ylabel('log regularization strength')
plt.title('Boston test accuracy')
plt.show()